In [1]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score

# Part 1: Kernel functions

Implement the function `poly_kernel` that defines the polynomial kernel with form $$k(x_1, x_2) = (1+x_1 \cdot x_2)^p,$$ where $p$ is the degree of the polynomial. Notice we are taking the dot product between two training samples, and that when $p=1$, we have the linear kernel.

*Hint:* you may find `np.sum` or `np.power` function helpful

In [8]:
#grade

def poly_kernel(x1, x2, degree):
    # x1, x2 feature vectors with shape (d,)
    # implement the polynomial kernel function given the degree

    ### START YOUR CODE ###
    #print((1 + np.inner(x1, x2))**degree)
    return (1 +np.inner(x1, x2))**degree
    
    ### END CODE ###

Implement the function `rbf_kernel` that defines the RBF (Gaussian) kernel given parameter $\sigma$ with form $$k(x_1, x_2) = \exp(-\frac{\lVert x_1-x_2 \rVert^2}{2 \sigma^2}).$$
*Hint:* you may find `np.exp` or `la.norm` function helpful

In [20]:
#grade

def rbf_kernel(x1, x2, sigma):
    # x1, x2 feature vectors with shape (d,)
    # implement the rbf (Gaussian) kernel function given sigma

    ### START YOUR CODE ###
    return np.exp( -la.norm(x1 - x2)**2 / ( 2* (sigma**2) ) )

    
    ### END CODE ###

In [21]:
#SAMPLE TEST CASE
x1 = np.array([1, 2, 3])
x2 = np.array([3, 4, 5])
assert poly_kernel(x1, x2, 1) == 27
assert poly_kernel(x1, x2, 2) == 729
assert rbf_kernel(x1, x2, 1).round(4) == 0.0025
assert rbf_kernel(x1, x2, 2).round(4) == 0.2231

# Part 2: Gram matrix

Implement the function `Gram`, that creates the Gram matrix for a given kernel function. The two functions provided below (`poly` and `rbf`) each return their corresponding kernel function given the polynomial degree or coefficient $\sigma$. The `Gram` function takes as input such a function `kernel`, that is already initialized with either a degree (in the polynomial case) or $\sigma$ (in the Gaussian case). In particular, `kernel` behaves as the functions you implemented above without the coefficient parameters (i.e., they directly follow the given mathematical definitions, and only take 2 training samples as input and return a scalar).

In [22]:
def poly(degree):
    return lambda x1, x2: poly_kernel(x1, x2, degree)

In [23]:
def rbf(sigma):
    return lambda x1, x2: rbf_kernel(x1, x2, sigma)

In [37]:
#grade

def Gram(x_train, kernel):
  # x_train: 2d array with shape (n, d)
  # kernel: The kernel function
    n, d = x_train.shape
  # implement the Gram matrix given the training set and a kernel function
    G = np.empty((n,n))
  ### START YOUR CODE ###
    for i in range(n):
        
        for j in range(n):
            
            G[i][j] = kernel(x_train[i,:], x_train[j,:])
            
    return G

  ### END CODE ###

In [38]:
#SAMPLE TEST CASE
x_train = np.array([[1, 2, 3], [3, 4, 5], [3, 0, 2], [4, 1, 2]])
k_poly = poly(2)
k_rbf = rbf(2)

print(Gram(x_train, k_poly).round(2))
assert np.array_equal(Gram(x_train, k_poly).round(2), np.array([[ 225.,  729.,  100.,  169.],
                                                      [ 729., 2601.,  400.,  729.],
                                                      [ 100.,  400.,  196.,  289.],
                                                      [ 169.,  729.,  289.,  484.]]).round(2))

assert np.array_equal(Gram(x_train, k_rbf).round(2), np.array([[1.        , 0.22313016, 0.32465247, 0.2528396 ],
                                                               [0.22313016, 1.        , 0.04393693, 0.09301449],
                                                               [0.32465247, 0.04393693, 1.        , 0.77880078],
                                                               [0.2528396 , 0.09301449, 0.77880078, 1.        ]]).round(2))

[[ 225.  729.  100.  169.]
 [ 729. 2601.  400.  729.]
 [ 100.  400.  196.  289.]
 [ 169.  729.  289.  484.]]


# Part 3: Kernalized Dual Hard-Margin SVM (from scratch)


Recall that if the data is 
- linearly separable, we use hard-margin SVM **(Lab 6)**.
- linearly non-separable, we use soft-margin SVM.

Now we have the kernel trick, then if the data is
- non-linearly separable, we use hard-margin kernerlized SVM **(Lab 7)**.
- non-linearly non-separable, we use soft-margin kernerlized SVM. 

Intuitively, if the data $X\in \mathbb{R}^{n\times d}$ is not linearly separable, we can use a function $\phi:\mathbb{R}^{d}→\mathbb{R}^p$ so that each data point $x\in\mathbb{R}^d$ can be mapped into a higher dimensional space. Then the data may become linear separable in the higher dimensional space.

However, such mapping will lead to much higher computational cost if the $p$ is large and the computation is carried out explicitly in the higher dimensional space. In the dual form of identifying linear classifiers, for example in the case of SVM, since the key computational elements can be reduced to computing inner products in the higher dimensional space, we can use the kernel trick to eliminate such costs. 

## 3(a): Lagrange Dual Problem

Remember from class, the Lagrange dual problem of hard-margin SVM is: 
$$\begin{align}
\mathcal{L}_d (\alpha)
&= \sum_{i=0}^n \alpha_i - \frac 12 \sum_{i=0}^n \sum_{k=0}^n \alpha_i \alpha_k y_i y_k \phi(x_i)^T \phi(x_k) \\
\end{align}$$

__Subject to $\forall i\in 1..n$:__
- $0 \le \alpha_i$
- $\sum_{i=0}^n \alpha_i y_i = 0$

In [ ]:
from scipy import optimize

In [41]:
#grade

def Ld(G, alpha, y):
    '''
    Parameters:
      G: Gram Matrix (n, n)
      alpha: Lagrangian Multiplier (n, )
      y: has shape (n, ) and has class labels {-1, 1}
    Return:
      Lagrange dual objective of hard-margin SVM, A Scalar
    '''
    ### START YOUR CODE ###
    
    t1 = np.sum(alpha)
    
    n = alpha.shape[0]
    
    t2 = 0
    
    for i in range(n):
        
        s = 0
        
        for k in range(n):
            
            s += alpha[i] * alpha[k] * y[i] * y[k] * G[i][k]
            
            
        t2 += s
            
    
    return t1 - t2/2

    
    ### END CODE ###

In [42]:
# Sample Test Case
np.random.seed(1234)
X = np.array([[1, 2, 3], [3, 4, 5], [3, 0, 2], [4, 1, 2]])
y = np.array([-1, 1, -1, 1])
alpha = np.array([0.1, 0.4, 0.3, 0.2])
assert np.round(Ld(Gram(X, poly(2)), alpha, y), 2) == -190.15

## 3(b): Train (You will not write any code in this section)
Now, our objective becomes to maximize $\mathcal{L}_d (\alpha)$. 

In Section 6.8 from the Ng-Ma Notes, we can use the Sequential Minimal Optimization (SMO) algorithm. But here we simply use the `optimize` function from `scipy` to compute $\alpha$. The function returns only the *support vectors*, namely those for which $\alpha_i > 0$.

In [43]:
def train_Ld(kernel, X, y, C):
    N = len(y)
    G = Gram(X, kernel)  
    yp = y.reshape(-1, 1)
    GramHXy = G * np.matmul(yp, yp.T) 

    def Ld0dAlpha(G, alpha):
        return np.ones_like(alpha) - alpha.dot(G)

    A = np.vstack((-np.eye(N), np.eye(N)))
    b = np.hstack((np.zeros(N), C * np.ones(N)))
    constraints = ({'type': 'eq',   'fun': lambda a: np.dot(a, y),     'jac': lambda a: y},
                    {'type': 'ineq', 'fun': lambda a: b - np.dot(A, a), 'jac': lambda a: -A})

    optRes = optimize.minimize(fun=lambda a: -Ld(G, a, y),
                                x0=np.ones(N), 
                                method='SLSQP', 
                                jac=lambda a: -Ld0dAlpha(GramHXy, a), 
                                constraints=constraints)
    alpha = optRes.x
    epsilon = 1e-8
    supportIndices = alpha > epsilon
    supportVectors = X[supportIndices]
    supportY = y[supportIndices]
    supportAlpha = alpha[supportIndices]
    return supportVectors, supportY, supportAlpha

## 3(c): Test
Since $ w = \sum_{i=0}^n \alpha_i y_i \phi(x_i)$, the prediction function is now : 
$$ f(x) = sign(w^T \phi(x) + b) = sign \left(\sum_{i=0}^n \alpha_i y_i \langle \phi(x_i), \phi(x) \rangle \right) $$

Recall that that `train_Ld` function returns only the support vectors, i.e., those for which $\alpha_i > 0$. For indices $i$ that are not in the support vector, $\alpha_i$ is $0$ and hence those terms don't need to be added in the about equation while predicting.

In [112]:
#grade

def predict(X, k, sV, sY, sA):
    """ Predict y values in {-1, 1} """
    '''
    Parameters:
      X: Test set, (n1, d)
      k: kernel function, k(x, z)
      sV: support vectors got after training, (?, d)
      sY: support vector labels, (?,)
      sA: corresponding alphas for support vectors, (?, d)
    Return:
      y_hat: predict results, (n1,)
    '''
    y_hat = []
    ### START YOUR CODE ###
    
    n = sV.shape[0]
    s = 0
    print(X.shape)
    print(sV.shape)

    for i in range(n):

        s += sA[i] * sY[i] * k(sV[i,:], X)
        
 
    y_hat.append(np.sign(s))
        
    
        

    ### END CODE ###
    return np.array(y_hat, dtype = np.int64)

In [113]:
# Sample Test Case
np.random.seed(1234)
sV = np.array([[1, 2, 3], [3, 4, 5], [3, 0, 2], [4, 1, 2]])
sY = np.array([-1, -1, -1, 1])
sA = np.array([0.1, 0.4, 0.3, 0.2])
X_test = np.array([[9, 9, 9]])
print(predict(X_test, poly(2), sV, sY, sA))
assert predict(X_test, poly(2), sV, sY, sA) == -1

(1, 3)
(4, 3)
[[-1]]
(1, 3)
(4, 3)


## 3(d): Application (You will not write any code in this section.)

Now, let's try our SVM on a non-linear XOR dataset. We will do this again later, but using sklearn on a more complicated dataset.

### Helper Functions

In [ ]:
def generateBatchXor(n, mu=0.5, sigma=0.5):
    """ Four gaussian clouds in a Xor fashion """
    X = np.random.normal(mu, sigma, (n, 2))
    yB0 = np.random.uniform(0, 1, n) > 0.5
    yB1 = np.random.uniform(0, 1, n) > 0.5
    # y is in {-1, 1}
    y0 = 2. * yB0 - 1
    y1 = 2. * yB1 - 1
    X[:,0] *= y0
    X[:,1] *= y1
    X -= X.mean(axis=0)
    return X, y0*y1

import matplotlib.colors as pltcolors

colors = ['blue','red']
cmap = pltcolors.ListedColormap(colors)

def plotLine(ax, xRange, w, x0, label, color='grey', linestyle='-', alpha=1.):
    """ Plot a (separating) line given the normal vector (weights) and point of intercept """
    if type(x0) == int or type(x0) == float or type(x0) == np.float64:
        x0 = [0, -x0 / w[1]]
    yy = -(w[0] / w[1]) * (xRange - x0[0]) + x0[1]
    ax.plot(xRange, yy, color=color, label=label, linestyle=linestyle)
    
def plotSvm(X, y, support=None, w=None, intercept=0., label='Data', separatorLabel='Separator', 
            ax=None, bound=[[-1., 1.], [-1., 1.]]):
    """ Plot the SVM separation, and margin """
    if ax is None:
        fig, ax = plt.subplots(1)
    
    im = ax.scatter(X[:,0], X[:,1], c=y, cmap=cmap, alpha=0.5, label=label)
    if support is not None:
        ax.scatter(support[:,0], support[:,1], label='Support', s=80, facecolors='none', 
                   edgecolors='y', color='y')
        print("Number of support vectors = %d" % (len(support)))
    if w is not None:
        xx = np.array(bound[0])
        plotLine(ax, xx, w, intercept, separatorLabel)
        # Plot margin
        if support is not None:
            signedDist = np.matmul(support, w)
            margin = np.max(signedDist) - np.min(signedDist) * np.sqrt(np.dot(w, w))
            supportMaxNeg = support[np.argmin(signedDist)]
            plotLine(ax, xx, w, supportMaxNeg, 'Margin -', linestyle='-.', alpha=0.8)
            supportMaxPos = support[np.argmax(signedDist)]
            plotLine(ax, xx, w, supportMaxPos, 'Margin +', linestyle='--', alpha=0.8)
            ax.set_title('Margin = %.3f' % (margin))
    ax.legend(loc='upper left')
    ax.grid()
    ax.set_xlim(bound[0])
    ax.set_ylim(bound[1])
    cb = plt.colorbar(im, ax=ax)
    loc = np.arange(-1,1,1)
    cb.set_ticks(loc)
    cb.set_ticklabels(['-1','1'])

### Predict + Visualize

In [ ]:
np.random.seed(1234)
N = 100
X_train, y_train = generateBatchXor(2*N, sigma=0.25)
plotSvm(X_train, y_train)
X_test, y_test = generateBatchXor(2*N, sigma=0.25)

In [ ]:
kernel = poly(2)
sV, sY, sA = train_Ld(kernel, X_train, y_train, C=5)
y_hat = predict(X_test, kernel, sV, sY, sA)
print('Model accuracy score: {0:0.2f}%'. format(100*accuracy_score(y_test, y_hat)))

In [ ]:
kernel = rbf(1)
sV, sY, sA = train_Ld(kernel, X_train, y_train, C=5)
fig, ax = plt.subplots(1, figsize=(11, 7))
plotSvm(X_train, y_train, support=sV, label='Training', ax=ax)

# Estimate and plot decision boundary
xx = np.linspace(-1, 1, 50)
X0, X1 = np.meshgrid(xx, xx)
xy = np.vstack([X0.ravel(), X1.ravel()]).T
Y30 = predict(xy, kernel, sV, sY, sA).reshape(X0.shape)
ax.contour(X0, X1, Y30, colors='k', levels=[-1, 0], alpha=0.3, linestyles=['-.', '-']);

# Part 4: Kernelized SVM using `sklearn` (You will not write any code in this section.)

Library functions in sklearn are usually more efficient that code that you write from scratch. This part is for you to look at how these libraries can be used on some datasets and visualize those results. We will not be using the code you wrote in Part 3. 

------
Now that you have implemented some kernel functions, the Gram matrix and a kernalized SVM model, we are going to look at kernelized SVMs in different contexts and examine the advantages they bring. To do this, we are going to use the sklearn.svm module that includes SVM algorithms that support kernels. You can examine the documentation and source code of various algorithms [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm); they should look familiar to you between last week's SVM lab and your work in parts 1 and 2.

## The XOR dataset

A convenient and popular problem that is not linearly seperable is the XOR dataset, which is the truth table related to the XOR logical operator. Let's visualize the dataset below; confirm with yourself that this dataset is not linearly seperable.

In [ ]:
# create XOR data
def xor_data():
    x = np.array([[1, 1], [-1, 1], [-1, -1], [1, -1]])
    y = np.array([1, -1, 1, -1])
    return x, y

# visualize XOR data
x, y = xor_data()
plt.scatter(x[:,0], x[:,1], c = y)

In [ ]:
x, y = xor_data()
kernel = rbf(1)
sV, sY, sA = train_Ld(kernel, x, y, C=5)
y_hat = predict(x, kernel, sV, sY, sA)
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y, y_hat)))

Run the below code multiple times, that fits a linear SVM to the XOR dataset and then predicts on the XOR dataset. What do you notice? Can you explain the performance you are seeing?

In [ ]:
# instantiate linear classifier
svc=SVC(kernel = 'linear') 

# fit classifier to XOR data
svc.fit(x,y)

# make predictions on XOR data
y_pred=svc.predict(x)

plt.scatter(x[:,0], x[:,1], c = y_pred)

Now let's try with a polynomial kernel, with just degree 2. Go ahead and run this a few times as well; what can you say about the stability of the model? Can you explain the performance here?

In [ ]:
# instantiate polynomial classifier with degree 2
svc=SVC(kernel = 'poly', degree = 2) 

# fit classifier to XOR data
svc.fit(x,y)

# make predictions on XOR data
y_pred=svc.predict(x)

plt.scatter(x[:,0], x[:,1], c = y_pred)

Let's try with the rbf kernel. The kernel here uses an equivalent definition with parameter $\gamma = \frac{1}{2 \sigma^2}$.

In [ ]:
# instantiate rbf classifier with sigma = 1
svc=SVC(kernel = 'rbf', gamma = 0.5) # corresponds to sigma = 1

# fit classifier to XOR data
svc.fit(x,y)

# make predictions on XOR data
y_pred=svc.predict(x)

plt.scatter(x[:,0], x[:,1], c = y_pred)

Now that we have convinced ourselves that kernel methods do actually work and can indeed provide advantages with higher dimensional datasets, let's examine a more complex dataset.

**Remark**: You are welcome to check out this [website](https://jgreitemann.github.io/svm-demo). Set up some points and their classes, specify the kernel method, then see the actual SVM margins and hyperplane.

## MNIST dataset

Let's look at a subset of the MNIST dataset again, which we have used to evaluate algorithms in previous labs. Even with a small subset, the dataset is complex enough that different kernel functions have noticeable performance differences. You can run the variants below to see this, which will take some time. Feel free to step away and take a break while they run. Why do you think that, in a complex, "real" dataset, polynomial or rbf kernels do better than linear kernels?

In [ ]:
# load the training and testing data
train = pd.concat(map(pd.read_csv, ["train_1.csv","train_2.csv","train_3.csv","train_4.csv"]), ignore_index=True)
print(train.shape)
test = pd.concat(map(pd.read_csv, ["test_1.csv","test_2.csv"]), ignore_index=True)
print(test.shape)
train_labels = np.array(train)[:,0]
train_data = np.array(train)[:,1:]
test_labels = np.array(test)[:,0]
test_data = np.array(test)[:,1:]

In [ ]:
svc=SVC(kernel = 'linear')
# fit classifier to training data
svc.fit(train_data,train_labels)

# make predictions on testing data
y_pred=svc.predict(test_data)

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(test_labels, y_pred)))

In [ ]:
svc=SVC(kernel = 'poly', degree = 3)
# fit classifier to training data
svc.fit(train_data,train_labels)

# make predictions on testing data
y_pred=svc.predict(test_data)

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(test_labels, y_pred)))

In [ ]:
svc=SVC(kernel = 'rbf') # the default gamma value is "scaled". See documentation for details
# fit classifier to training data
svc.fit(train_data,train_labels)

# make predictions on testing data
y_pred=svc.predict(test_data)

print('Model accuracy score: {0:0.4f}'. format(accuracy_score(test_labels, y_pred)))